In [ ]:
import json, os

test_name = 'neuroM_morph_softChecks'
model_results = 'population_str-fs_excert/20180529-100621'
val_results_dir='./validation_results'
val_results_path = os.path.join(val_results_dir, test_name, model_results)

if not os.path.exists(val_results_path):
    os.makedirs(val_results_path)
    
val_file_path = os.path.join(val_results_path,'prediction_summary_FSI_pop.json')
with open(val_file_path, 'r') as fp:
    pop_prediction_raw = json.load(fp)

In [ ]:
import pandas as pd
prediction_raw = pop_prediction_raw.values()[0]

pred_axon_df = pd.DataFrame(prediction_raw['axon'])
pred_axon_df.insert(0, 'NeuriteType', 'axon')

pred_dend_df = pd.DataFrame(prediction_raw['basal_dendrite'])
pred_dend_df.insert(0, 'NeuriteType', 'basal_dendrite')

frames = [pred_axon_df, pred_dend_df]
prediction_raw_df = pd.concat(frames)
prediction_raw_df.index.rename('Cell', inplace=True)
prediction_raw_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
all_vars=list(prediction_raw_df)
all_vars.remove('total_number_of_neurite')
all_vars.remove('NeuriteType')
                
NeuriteType=['axon']
for neuriteType in NeuriteType:
    filepath = os.path.join(val_results_path,'prediction_stats_'+neuriteType+'_FSI_pop.pdf')
    # plt.figure()
    data=prediction_raw_df.loc[prediction_raw_df.NeuriteType==neuriteType]
    sns.pairplot(data, vars=all_vars, diag_kind="kde", kind='reg', markers="+",
              diag_kws=dict(shade=True))
    # plt.show()
    # plt.savefig(filepath, dpi=600, )
    
#######################
all_vars = ["max_section_branch_order", "neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent", 
          "neurite_field_diameter", "neurite_largest_extent", "neurite_shortest_extent", 
          "total_neurite_length", "total_number_of_neurite"]

NeuriteType=['basal_dendrite']
for neuriteType in NeuriteType:
    filepath = os.path.join(val_results_path,'prediction_stats_'+neuriteType+'_FSI_pop.pdf')
    # plt.figure()
    data=prediction_raw_df.loc[prediction_raw_df.NeuriteType==neuriteType]
    sns.pairplot(data, vars=all_vars, diag_kind="kde", kind='reg', markers="+",
              diag_kws=dict(shade=True))
    # plt.show()
    # plt.savefig(filepath, dpi=600, )

In [ ]:
y_vars=["neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent"]
x_vars=["total_neurite_length", "neurite_field_diameter", "neurite_largest_extent", "neurite_shortest_extent"]

#filepath = os.path.join(val_results_path,'statistics_FSI_pop.pdf')
# data=prediction_raw_df
# sns.pairplot(data, x_vars=x_vars, y_vars=y_vars, hue="NeuriteType", size=4, aspect=1, kind="reg")
# plt.savefig(filepath, dpi=600, )


from scipy import stats

def corrfunc(x, y, **kws):
    r,p = stats.pearsonr(x,y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}, p ={:.4f}".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)

NeuriteType=['axon', 'basal_dendrite']
for neuriteType in NeuriteType:
    filepath = os.path.join(val_results_path,'prediction_lreg_'+neuriteType+'_FSI_pop.pdf')
    data=prediction_raw_df.loc[prediction_raw_df.NeuriteType==neuriteType]
    g = sns.PairGrid(data, x_vars=x_vars, y_vars=y_vars, size=5, aspect=1, palette=["red"])
    g.map(sns.regplot)
    g.map(corrfunc)
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('NeuriteType: '+neuriteType, fontsize=20) 

    # sns.pairplot(data, x_vars=x_vars, y_vars=y_vars, hue="NeuriteType", size=5, aspect=1, kind="reg")
    plt.savefig(filepath, dpi=600, )

"""    
with sns.axes_style('white'):
    for x,y in zip(x_vars,y_vars):
        sns.jointplot(x, y, data.loc[data.NeuriteType=='axon'], kind='reg')
"""

In [ ]:
from scipy import stats

def corrfunc(x, y, **kws):
    r,p = stats.pearsonr(x,y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}, p ={:.2f}".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)

all_vars=list(prediction_raw_df)
all_vars.remove('max_section_branch_order')
all_vars.remove('total_number_of_neurite')
data_excerpt_df = prediction_raw_df.loc[:, all_vars]

NeuriteType=['axon', 'basal_dendrite']
for neuriteType in NeuriteType:
    filepath = os.path.join(val_results_path,'prediction_stats_'+neuriteType+'_FSI_pop.pdf')
    data=data_excerpt_df.loc[data_excerpt_df.NeuriteType==neuriteType]
    g = sns.PairGrid(data, palette=["red"])
    g.map_upper(sns.regplot)
    #g.map_upper(plt.reg, s=10)
    g.map_diag(sns.distplot, kde=False)
    g.map_lower(sns.kdeplot, cmap="Blues_d", n_levels=6)
    g.map_upper(corrfunc)
    plt.subplots_adjust(top=0.95)
    g.fig.suptitle('NeuriteType: '+neuriteType, fontsize=20) 
    plt.savefig(filepath, dpi=600, )

In [ ]:
### x_vars = ["max_section_branch_order", "neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent", 
          "neurite_field_diameter", "neurite_largest_extent", "neurite_shortest_extent", 
          "total_neurite_length", "total_number_of_neurite"]

NeuriteType=['basal_dendrite']
for neuriteType in NeuriteType:
    filepath = os.path.join(val_results_path,'prediction_stats_'+neuriteType+'_FSI_pop.pdf')
    plt.figure()
    data=prediction_raw_df.loc[prediction_raw_df.NeuriteType==neuriteType]
    g = sns.FacetGrid(prediction_raw_df, col='total_number_of_neurite', row='total_number_of_neurite')
    g.map(plt.plot, "neurite_largest_extent", "neurite_shortest_extent")
    
# col=["neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent"],
# row=["neurite_X_extent", "neurite_Y_extent", "neurite_Z_extent"]